In [40]:
# !pip install textblob
# !pip install textstat
# !pip install --upgrade textstat
# !pip uninstall textstat
# !pip install html5lib
# !pip install textstat==0.7.0
import requests                           #for making HTTP requests 
import openpyxl                           #for reading Excel files
from bs4 import BeautifulSoup             #for parsing HTML content
import numpy as np
import pandas as pd                       #for data manipulation and analysis
import regex as re                        #for regular expressions
import os                                 
import nltk                               # for natural language processing
# nltk.download('punkt')
from textblob import TextBlob             #for text sentiment analysis
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import textstat                            #for text statistics
from textstat.textstat import textstatistics,legacy_round

In [3]:
# ittrace over all rows of excel file and pass url to fun
# the function takes url and extracts the content 

def extract_article(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content.decode("utf-8"), "html.parser")
    title = soup.find("title").get_text(strip=True)
    paragraphs = soup.find_all("p", class_=None, id=None, style=None)
    paragraphs = [re.sub('<[^<]+?>', '', str(p)) for p in paragraphs]
    article_text = "\n".join(paragraphs)
    with open(f"{url_id}.txt", "w", encoding="utf-8") as f:
        f.write(f"{title}\n{article_text}")

if __name__ == "__main__":
    path = 'C:\\Users\\arif shaikh\\Downloads\\Input.xlsx'
    wb_obj = openpyxl.load_workbook(path)
    sheet_obj = wb_obj.active
    max_row = sheet_obj.max_row

    for i in range(2, max_row + 1):
        url_id = sheet_obj.cell(row=i, column=1).value
        cell_obj = sheet_obj.cell(row=i, column=2)
        url = cell_obj.value
        extract_article(url)
        
    wb_obj.close()

In [41]:
print(os.getcwd())

C:\Users\arif shaikh


In [42]:
# readline of all textfiles(line 1 is treated as title) 
# and store it to a list and later convert to dataframe with columns title and article 

folder_path = 'C:\\Users\\arif shaikh\\'
data = []

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.readlines()
            if len(content) >= 2:
                title = content[0].strip()
                article = ' '.join(content[1:]).strip()
                data.append((title, article))

df = pd.DataFrame(data, columns=['Title', 'Article'])


In [43]:
# replace , by whitespace
df['Article']=df['Article'].str.replace(',','')
df.shape


(112, 2)

In [44]:
df.tail()

,Title,Article
107,Coronavirus impact on energy markets | Blackco...,The coronavirus also known as COVID-19 is not ...
108,COVID-19 Impact on Hospitality Industry | Blac...,In December 2019 a novel coronavirus strain (S...
109,Lessons from the past: Some key learnings rele...,As a race we are not ready for any kind of cri...
110,Estimating the impact of COVID-19 on the world...,COVID-19 an unprecedented pandemic for us but ...
111,,greeley\n cuellar\n unfortunately\n guerin\n c...


In [6]:
# since there are multiple stopfiles well combine them together

Stop_folder = r'C:\Users\arif shaikh\Downloads\Stop_folder\\'  
output_file = 'combined_stopwords.txt'  

combined_stop_words = set()  #to avoid duplicates

for filename in os.listdir(Stop_folder):
    if filename.endswith('.txt'):
        file_path = os.path.join(Stop_folder, filename)
        with open(file_path, 'r', encoding='latin-1') as file:
            stop_words = file.read().lower().split('\n')
            combined_stop_words.update(stop_words)

with open(output_file, 'w', encoding='utf-8') as file:
    file.write('\n'.join(combined_stop_words))

print(f"Combined stop words saved to {output_file}.")


Combined stop words saved to combined_stopwords.txt.


In [7]:
stopWordsFile =r'C:\Users\arif shaikh\combined_stopwords.txt'
positiveWordsFile ='C:\\Users\\arif shaikh\\Downloads\\positive-words.txt'
nagitiveWordsFile ='C:\\Users\\arif shaikh\\Downloads\\negative-words.txt'

In [66]:
# filter out words whivh are not in stopwords

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
    stopWordList = stopWords.split('\n')
    stopWordList[-1:] = []            #removes the last element(mostly emptyspace)
    
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+') #RegexpTokenizer object that tokenizes the text based on expression
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words  


In [67]:
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
    positiveWordList=positivewords.split('\n')

def positive_score(text):
    PosWords = 0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in positiveWordList:
            PosWords  += 1
    
    sumPos = PosWords
    return sumPos

In [68]:
with open(nagitiveWordsFile ,'r') as negfile:
    negativeword=negfile.read().lower()
    negativeWordList=negativeword.split('\n')
    
def negative_score(text):
    NegWords=0
    rawToken = tokenizer(text)
    for word in rawToken:
        if word in negativeWordList:
            NegWords -=1
    sumNeg = NegWords 
    sumNeg = sumNeg * -1
    return sumNeg

In [69]:
df["positive_score"] = df["Article"].apply(positive_score)
df["negative_score"] = df["Article"].apply(negative_score)

In [70]:
polarity = []

for i in df["Article"]:
    polarity.append(TextBlob(i).sentiment.polarity)

df["polarity_score"] = polarity

In [71]:
subjectivity = []

for i in df["Article"]:
    subjectivity.append(TextBlob(i).sentiment.subjectivity)
    
df["subjectivity_score"] = subjectivity

In [72]:
def average_sentence_length(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

df["Average sentence length"] = df["Article"].apply(average_sentence_length)


In [73]:
#  difficult or advanced words in terms of their linguistic complexity
# if it has more than two vowels and does not end with 'es' or 'ed'.

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

df["Percentage complex words"] = df["Article"].apply(percentage_complex_word)
df["Percentage complex words"] = df["Percentage complex words"].astype(float)


In [74]:
# measure of readability or textual complexity
def fog_index(average_sentence_length, percentage_complex_word):
    fogIndex = 0.4 * (average_sentence_length + percentage_complex_word)
    return fogIndex

df["Fog index"] = np.vectorize(fog_index)(df['Average sentence length'],df['Percentage complex words'])
# element-wise to the corresponding 

In [75]:
def average_wordper_sent(text):
    sentence_list = sent_tokenize(text)
    tokens = tokenizer(text)
    totalWord = len(tokens)
    totalSent = len(sentence_list)
    if totalSent != 0:
        average_word_sent = totalWord / totalSent
    
    return round(average_word_sent)

df["Avg words per sentence"] = df["Article"].apply(average_wordper_sent)


In [76]:
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):    #non complex
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

df["Complex word count"] = df["Article"].apply(complex_word_count)


In [77]:
import string
def word_count(text):
    tokens = tokenizer(text)
    tokens = [word for word in tokens if word not in string.punctuation]
    return len(tokens)

df["Word Count"] = df["Article"].apply(word_count)


In [78]:
def syllables(word):
    vowels = ['a', 'e', 'i', 'o', 'u']
    syllable_count = 0
    
    if word.endswith(('es', 'ed')):
        word = word[:-2]
    
    for i in range(len(word)):
        if word[i] in vowels and (i == 0 or word[i-1] not in vowels):
            syllable_count += 1
    
    return syllable_count

def syllables_per_word(text):
    tokens = tokenizer(text)
    word_count = len(tokens)
    
    total_syllables = sum(syllables(word) for word in tokens)
    SPW = total_syllables / word_count
    
    return round(SPW, 1)

df["Syllable per word"] = df["Article"].apply(lambda x: syllables_per_word(x))


In [79]:
def pronouns(text):
    count = 0
    pronoun_list = ['I', 'we', 'my', 'ours', 'us', 'We', 'My', 'Ours', 'Us']
    for word in re.findall(r'\b\w+\b', text):
        if word in pronoun_list:
            count += 1
    return count

df["Personal pronoun"] = df["Article"].apply(pronouns)


In [80]:
def avg_word_len(text):
    char_list = textstat.char_count(text, ignore_spaces=True)
    tokens = tokenizer(text)
    totalWord = len(tokens)
    totalchar = (char_list)
    if totalWord!= 0:
        avg_char = totalchar / totalWord
    
    return round(avg_char)

df["Average word length"] = df['Article'].apply(avg_word_len)


In [60]:
des = r'C:\Users\arif shaikh\Downloads\Input.xlsx'
df1 = pd.read_excel(des)
df1.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [81]:
fin_df = pd.concat([df1,df],axis = 1)
fin_df.head()

,URL_ID,URL,Title,Article,positive_score,negative_score,polarity_score,subjectivity_score,Average sentence length,Percentage complex words,Avg words per sentence,Complex word count,Word Count,Syllable per word,Personal pronoun,Average word length,Fog index
0,37,https://insights.blackcoffer.com/ai-in-healthc...,Estimating the impact of COVID-19 on the world...,Will COVID19 END Globalization?\n Globalizatio...,21.0,42.0,0.078863,0.330512,11.0,0.364063,11.0,233.0,640.0,2.1,3.0,10.0,4.545625
1,38,https://insights.blackcoffer.com/what-if-the-c...,Travel and Tourism Outlook | Blackcoffer Insights,The UN projects a 20-30% decline in internatio...,3.0,3.0,0.273255,0.424912,7.0,0.328358,7.0,22.0,67.0,1.9,2.0,13.0,2.931343
2,39,https://insights.blackcoffer.com/what-jobs-wil...,Gaming Disorder and Effects of Gaming on Healt...,Perhaps the virtual illusion has become today’...,24.0,42.0,0.061629,0.433699,8.0,0.357895,8.0,170.0,475.0,2.1,13.0,11.0,3.343158
3,40,https://insights.blackcoffer.com/will-machine-...,What is the repercussion of the environment du...,What is COVID 19 pandemic?\n On 31st December ...,6.0,24.0,0.059480,0.405852,9.0,0.331169,9.0,102.0,308.0,2.0,4.0,10.0,3.732468
4,41,https://insights.blackcoffer.com/will-ai-repla...,Due to the COVID-19 the repercussion of the en...,Epidemics in general have both direct and indi...,25.0,54.0,0.057154,0.391464,11.0,0.388686,11.0,213.0,548.0,2.3,2.0,11.0,4.555474


In [82]:
del fin_df["Title"]

In [83]:
del fin_df["Article"]

In [84]:
fin_df.head()

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity_score,Average sentence length,Percentage complex words,Avg words per sentence,Complex word count,Word Count,Syllable per word,Personal pronoun,Average word length,Fog index
0,37,https://insights.blackcoffer.com/ai-in-healthc...,21.0,42.0,0.078863,0.330512,11.0,0.364063,11.0,233.0,640.0,2.1,3.0,10.0,4.545625
1,38,https://insights.blackcoffer.com/what-if-the-c...,3.0,3.0,0.273255,0.424912,7.0,0.328358,7.0,22.0,67.0,1.9,2.0,13.0,2.931343
2,39,https://insights.blackcoffer.com/what-jobs-wil...,24.0,42.0,0.061629,0.433699,8.0,0.357895,8.0,170.0,475.0,2.1,13.0,11.0,3.343158
3,40,https://insights.blackcoffer.com/will-machine-...,6.0,24.0,0.059480,0.405852,9.0,0.331169,9.0,102.0,308.0,2.0,4.0,10.0,3.732468
4,41,https://insights.blackcoffer.com/will-ai-repla...,25.0,54.0,0.057154,0.391464,11.0,0.388686,11.0,213.0,548.0,2.3,2.0,11.0,4.555474


In [86]:
fin_df.to_csv('TextAnalysisOutput1.csv',index = None)
# TextAnalysisOutput

Thanks a lot Black Coffere for your time and effort in organizing this task. It has been a great experience and I look forward to applying the skills I have learned to future projects.